In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.dataset import  OurRadiologsitsDataset
from data.dataset import collate_fn
from utils.transforms import get_transform
from our_radiologist.load import get_anns
from utils.coco_eval import get_ar_ap
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from utils.data import get_datasets, get_dataloaders

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

c:\Users\mike8\Desktop\multimodal-abnormalities-detection\models\rcnn.py:92: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
XAMI_MIMIC_PATH =  "D:\XAMI-MIMIC"

labels_cols = [
    "Enlarged cardiac silhouette",
    "Atelectasis",
    "Pleural abnormality",
    "Consolidation",
    "Pulmonary edema",
    #  'Groundglass opacity', # 6th disease.
]

normal_iou_thrs = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])

all_range_iou_thrs = np.array(
        [
            0.00,
            0.05,
            0.1,
            0.15,
            0.2,
            0.25,
            0.3,
            0.35,
            0.4,
            0.45,
            0.5,
            0.55,
            0.6,
            0.65,
            0.7,
            0.75,
            0.8,
            0.85,
            0.9,
            0.95,
            1.0,
        ]
    )

# Run evaluation.

In [4]:
for select_model in TrainedModels:

    model, train_info = get_trained_model(
        select_model,
        labels_cols,
        device,
        include_train_info=True,
        image_size=512,
        rpn_nms_thresh=0.3,
        box_detections_per_img=10,
        box_nms_thresh=0.2,
        rpn_score_thresh=0.0,
        box_score_thresh=0.05,
    )

    model.eval()

    dataset_params_dict = {
        "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        "with_clinical": train_info.model_setup.use_clinical,
        "dataset_mode": train_info.model_setup.dataset_mode,
        "bbox_to_mask": True,
        "labels_cols": [
            "Enlarged cardiac silhouette",
            "Atelectasis",
            "Pleural abnormality",
            "Consolidation",
            "Pulmonary edema",
            #  'Groundglass opacity', # 6th disease.
        ],
    }

    detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
        dataset_params_dict=dataset_params_dict
    )

    train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
        train_dataset, val_dataset, test_dataset, batch_size=2
    )

    radiologists_ann = get_anns("radiologists_annotated", detect_eval_dataset)

    radiologist_dataset = OurRadiologsitsDataset(detect_eval_dataset, radiologists_ann)

    radiologist_dataloader = torch.utils.data.DataLoader(
        radiologist_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn,
    )

    normal_eval_params_dict = get_eval_params_dict(
        detect_eval_dataset, iou_thrs=normal_iou_thrs,
    )

    all_range_eval_params_dict = get_eval_params_dict(
        detect_eval_dataset, iou_thrs=all_range_iou_thrs,
    )

    model.roi_heads.score_thresh = 0.05

    all_cat_ids = [None] + [
        detect_eval_dataset.disease_to_idx(d) for d in detect_eval_dataset.labels_cols
    ]

    for cat_id in all_cat_ids:
        cat_ids = (
            [
                detect_eval_dataset.disease_to_idx(d)
                for d in detect_eval_dataset.labels_cols
            ]
            if cat_id is None
            else [cat_id]
        )
        if not (cat_ids is None):
            normal_eval_params_dict["bbox"].catIds = cat_ids
            normal_eval_params_dict["segm"].catIds = cat_ids

        train_evaluator = xami_evaluate(
            model,
            train_dataloader,
            device=device,
            params_dict=normal_eval_params_dict,
            # score_thres=score_thres,
        )

        test_evaluator = xami_evaluate(
            model,
            test_dataloader,
            device=device,
            params_dict=normal_eval_params_dict,
            # score_thres=score_thres,
        )

        val_evaluator = xami_evaluate(
            model,
            val_dataloader,
            device=device,
            params_dict=normal_eval_params_dict,
            # score_thres=score_thres,
        )

        radiologist_evaluator = xami_evaluate(
            model,
            radiologist_dataloader,
            device=device,
            params_dict=normal_eval_params_dict,
        )

        if cat_id is None:
            disease_str = "all"
        else:
            disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

        train_ar, train_ap = get_ar_ap(
            train_evaluator, areaRng="all", maxDets=10, iouThr=None
        )

        test_ar, test_ap = get_ar_ap(
            test_evaluator, areaRng="all", maxDets=10, iouThr=None
        )
        val_ar, val_ap = get_ar_ap(
            val_evaluator, areaRng="all", maxDets=10, iouThr=None
        )
        our_ar, our_ap = get_ar_ap(
            radiologist_evaluator, areaRng="all", maxDets=10, iouThr=None
        )

        df = pd.DataFrame(
            [
                {
                    "dataset": "train",
                    f"AP@[IoBB = 0.50:0.95]": train_ap,
                    f"AR@[IoBB = 0.50:0.95]": train_ar,
                },
                {
                    "dataset": "test",
                    f"AP@[IoBB = 0.50:0.95]": test_ap,
                    f"AR@[IoBB = 0.50:0.95]": test_ar,
                },
                {
                    "dataset": "val",
                    f"AP@[IoBB = 0.50:0.95]": val_ap,
                    f"AR@[IoBB = 0.50:0.95]": val_ar,
                },
                {
                    "dataset": "our",
                    f"AP@[IoBB = 0.50:0.95]": our_ap,
                    f"AR@[IoBB = 0.50:0.95]": our_ar,
                },
            ]
        )

        df.to_csv(
            os.path.join("eval_results", f"{select_model.value}_{disease_str}.csv")
        )
        print_title(disease_str)
        print(df)

    for cat_id in all_cat_ids:

        cat_ids = (
            [
                detect_eval_dataset.disease_to_idx(d)
                for d in detect_eval_dataset.labels_cols
            ]
            if cat_id is None
            else [cat_id]
        )
        if not (cat_ids is None):
            all_range_eval_params_dict["bbox"].catIds = cat_ids
            all_range_eval_params_dict["segm"].catIds = cat_ids

        train_evaluator = xami_evaluate(
            model,
            train_dataloader,
            device=device,
            params_dict=all_range_eval_params_dict,
            # score_thres=score_thres,
        )

        test_evaluator = xami_evaluate(
            model,
            test_dataloader,
            device=device,
            params_dict=all_range_eval_params_dict,
            # score_thres=score_thres,
        )

        val_evaluator = xami_evaluate(
            model,
            val_dataloader,
            device=device,
            params_dict=all_range_eval_params_dict,
            # score_thres=score_thres,
        )

        radiologist_evaluator = xami_evaluate(
            model,
            radiologist_dataloader,
            device=device,
            params_dict=all_range_eval_params_dict,
        )

        if cat_id is None:
            disease_str = "all"
        else:
            disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

        save_iou_results(train_evaluator, f"train_{disease_str}", select_model.value)
        save_iou_results(test_evaluator, f"test_{disease_str}", select_model.value)
        save_iou_results(val_evaluator, f"val_{disease_str}", select_model.value)
        save_iou_results(
            radiologist_evaluator, f"our_{disease_str}", select_model.value
        )



Load custom model
Using ResNet50 as backbone
Using pretrained model
creating index...
index created!
creating index...
index created!
creating index...
index created!
Test:  [  0/207]  eta: 0:09:37  model_time: 2.2095 (2.2095)  evaluator_time: 0.1540 (0.1540)  time: 2.7876  data: 0.2931  max mem: 755
Test:  [100/207]  eta: 0:01:09  model_time: 0.0770 (0.1166)  evaluator_time: 0.1100 (0.1778)  time: 0.5820  data: 0.3156  max mem: 1343
Test:  [200/207]  eta: 0:00:04  model_time: 0.0800 (0.1062)  evaluator_time: 0.1090 (0.1791)  time: 0.5850  data: 0.2993  max mem: 1409
Test:  [206/207]  eta: 0:00:00  model_time: 0.0840 (0.1058)  evaluator_time: 0.1100 (0.1791)  time: 0.5930  data: 0.2995  max mem: 1409
Test: Total time: 0:02:11 (0.6370 s / it)
Averaged stats: model_time: 0.0840 (0.1058)  evaluator_time: 0.1100 (0.1791)
Accumulating evaluation results...
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.07s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.95 | area=  